# Memory mechanism

Testing on electronics only

Its job:
1. push data to it
2. push it to a chain
3. ask the chain to summarize

In [5]:
import sys
sys.path.append('../')
from botcore.setup import get_ai21_model, enable_tracing
from botcore.chains.qa_feature import build_ask_feature_chain
from botcore.chains.qa_elec_condition import build_ask_electronic_condition_chain

MODEL = get_ai21_model(max_tokens = 400)
enable_tracing("mem_mecha")

ask_feature = build_ask_feature_chain(MODEL)
ask_condition = build_ask_electronic_condition_chain(MODEL)

AI21 ready
Enable tracing at mem_mecha


In [6]:
product = "laptop"
n = 3

features = ask_feature({"product": product, "n_top": n})
features

{'product': 'laptop',
 'n_top': 3,
 'result': '\n\n```json\n[\n {\n  "question": "What is the screen size?",\n  "options": [\n   "14 inch",\n   "15.6 inch",\n   "13.3 inch",\n   "17.3 inch",\n   "11.6 inch"\n  ]\n },\n {\n  "question": "What is the RAM size?",\n  "options": [\n   "4 GB",\n   "8 GB",\n   "16 GB",\n   "32 GB",\n   "64 GB"\n  ]\n },\n {\n  "question": "What is the processor speed?",\n  "options": [\n   "2.5 GHz",\n   "3.0 GHz",\n   "2.0 GHz",\n   "1.5 GHz",\n   "1.0 GHz"\n  ]\n }\n]\n```'}

In [7]:
conditions = ask_condition({"product": product, "n_top": n})
conditions

{'product': 'laptop',
 'n_top': 3,
 'result': '\n\n```json\n[\n  {\n    "question": "What are the noticeable malfunctions?",\n    "options": [\n      "The laptop does not power on.",\n      "The laptop is very slow.",\n      "The laptop has missing keys."\n    ]\n  },\n  {\n    "question": "What are the physical damages?",\n    "options": [\n      "The laptop is cracked.",\n      "The laptop is dented.",\n      "The laptop is scratched."\n    ]\n  },\n  {\n    "question": "Is the warranty valid?",\n    "options": [\n      "Yes, the warranty is valid.",\n      "No, the warranty is invalid.",\n      "I am not sure."\n    ]\n  }\n]\n\n```'}

# Test

In [33]:
from botcore.utils.json_parser import parse_nested_json

feats = parse_nested_json(features['result'])
conds = parse_nested_json(conditions['result'])

def get_sample_qa(feats):
    ques_feat = [f['question'] for f in feats]
    ans_feat = [f['options'][0] for f in feats]
    return ques_feat, ans_feat

ques_feat, ans_feat = get_sample_qa(feats)
ques_cond, ans_cond = get_sample_qa(conds)
print(ques_cond)

['What are the noticeable malfunctions?', 'What are the physical damages?', 'Is the warranty valid?']


In [34]:
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain, PromptTemplate

def load_to_memory(ques_list, ans_list):
    for q, a in zip(ques_list, ans_list):
        memory.chat_memory.add_ai_message(q)
        memory.chat_memory.add_user_message(a)
    return True


template = """You are a secondhand dealer and having a conversation with a customer.

{chat_history}
Human: {human_input}
Answer:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")
memory.chat_memory.add_user_message(f"I have this used {product} of mine. Please ask me some questions about it.")
load_to_memory(ques_feat, ans_feat)
load_to_memory(ques_cond, ans_cond)
talk_chain = LLMChain(llm=MODEL, prompt=prompt, verbose=True, memory=memory)

In [37]:
# memory.chat_memory.messages
ans = talk_chain({"human_input": "What is my product?"})
ans



> Entering new  chain...
Prompt after formatting:
You are a secondhand dealer and having a conversation with a customer.

Human: I have this used laptop of mine. Please ask me some questions about it.
AI: What is the screen size?
Human: 14 inch
AI: What is the RAM size?
Human: 4 GB
AI: What is the processor speed?
Human: 2.5 GHz
AI: What are the noticeable malfunctions?
Human: The laptop does not power on.
AI: What are the physical damages?
Human: The laptop is cracked.
AI: Is the warranty valid?
Human: Yes, the warranty is valid.
Human: Do you think that my laptop will be able to sell at a good price?
AI: 
The laptop's screen size is 14 inches, the RAM size is 4 GB, the processor speed is 2.5 GHz, and it does not power on. The laptop is cracked, and the warranty is valid. The laptop may be able to sell, but at a lower price due to its malfunctions and damages.
Human: What is my product?
Answer:

> Finished chain.


{'human_input': 'What is my product?',
 'chat_history': "Human: I have this used laptop of mine. Please ask me some questions about it.\nAI: What is the screen size?\nHuman: 14 inch\nAI: What is the RAM size?\nHuman: 4 GB\nAI: What is the processor speed?\nHuman: 2.5 GHz\nAI: What are the noticeable malfunctions?\nHuman: The laptop does not power on.\nAI: What are the physical damages?\nHuman: The laptop is cracked.\nAI: Is the warranty valid?\nHuman: Yes, the warranty is valid.\nHuman: Do you think that my laptop will be able to sell at a good price?\nAI: \nThe laptop's screen size is 14 inches, the RAM size is 4 GB, the processor speed is 2.5 GHz, and it does not power on. The laptop is cracked, and the warranty is valid. The laptop may be able to sell, but at a lower price due to its malfunctions and damages.",
 'text': ' A used laptop'}